block 1 to 6 is api integration

In [1]:
import pandas as pd
import tkinter as tk
import tkinter as tk
from tkinter import ttk
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import datetime     
import secret
from datetime import date
import ast
import schedule
import time,datetime


In [2]:
api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)

In [ ]:
print(kite.login_url())

In [4]:
data = kite.generate_session("0LkrVxAuOaRh2n3VknyFV6F6sSH2ACcn", api_secret=api_secret)
kite.set_access_token(data["access_token"])
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)
print(data["access_token"])

LXm902ibb8Mc0vujQfg8EdlphN3hahtE


CALCULATE ALL THE LEVELS OF THE STOCKS (SWING HIGH AND SWING LOW)

In [5]:
def swingTop_bottom(monthly_data):
    data=[]
    length = len(monthly_data)

# n = int(inpucollectiont())  this will be used for specifying the counter for swing top
    for i in range(2,length-2):
        if ((monthly_data['high'][i] > monthly_data['high'][i+1]) and (monthly_data['high'][i] > monthly_data['high'][i+2]) and (monthly_data['high'] [i]> monthly_data['high'][i-1]) and (monthly_data['high'][i] > monthly_data['high'][i-2])):
            data.append(float(monthly_data['high'][i]))
         
        if ((monthly_data['low'][i] < monthly_data['low'][i+1]) and (monthly_data['low'][i] < monthly_data['low'][i+2]) and (monthly_data['low'][i] < monthly_data['low'][i-1]) and (monthly_data['low'][i] < monthly_data['low'][i-2])):
            data.append(float(monthly_data['low'][i]))
    

    return data

FUNCTION FOR CALCULATING THE NUMBER OF TAPS

In [6]:
def filterdTop(swingList,filteredata,levelAllowance):
    swingLen = len(swingList)
    filterdataLen = len(filteredata)
    # check=[]
    data ={}
    for i in range(swingLen):
        price =swingList[i]
        # print(price)
        percentage = ((price/100)*levelAllowance)
        subPercentage = price - percentage
        addPercentage = price + percentage
        # print(subPercentage,addPercentage)
        key = price
        # check.append('END')
        if key not in data:
            data[key]=0
        for j in range(filterdataLen):
            if ((float(filteredata['open'][j]) <= addPercentage) and (float(filteredata['open'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'open',float(filteredata['open'][j])])
            elif ((float(filteredata['high'][j]) <= addPercentage) and (float(filteredata['high'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'high',float(filteredata['high'][j])])
            elif ((float(filteredata['low'][j]) <= addPercentage) and (float(filteredata['low'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'low',float(filteredata['low'][j])])
            elif ((float(filteredata['close'][j]) <= addPercentage) and (float(filteredata['close'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'close',float(filteredata['close'][j])])
    
    return data

levels percentage

In [7]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge
       

function for first short listing of the stocks on the basis of 0.5%

In [8]:


def first_shortlist(symbol,addpercentage,subpercentage,ohlcCompare,close_data):
    stock_dict ={}
    stock_dict[symbol]=(set(),close_data)
    st = list(ohlcCompare.keys())
    close_add_percentage, close_sub_percentage = percentage(close_data, 4)
    for levels in range(len(st)):
       
        if (subpercentage <= float(st[levels]) <= addpercentage):
            
            if levels + 2 < len(st) and levels - 2 >= 0:
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels -1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels -2]]))
                
            elif levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                 

            elif levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]])) 

            elif levels - 1 >= 0 and levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                 
                

            elif levels + 1 < len(st) and levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]]))
                

   
    if stock_dict[symbol][0]:
        return stock_dict
    
    

this function take out the levels and there taps and this function will run at the end of the week

In [9]:
import json
watchlist ="FNO's"
timeframe = 'M'
instruments = kite.instruments("NSE")
nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
errorStockList=[]
levelTapping={}

In [ ]:


for symbols in nifty50['Symbol']:
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)
        to_date = datetime.date.today()
        from_date = "2019-10-01"
        data = kite.historical_data(instrument_token, from_date, to_date, "day")
        df = pd.DataFrame(data)
        # print(df)
        if (timeframe == 'D'):
            df['date'] = pd.to_datetime(df['date']).dt.date  #.dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
            monthly_data = df.groupby('date').agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).reset_index()
        else:
            df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
            monthly_data = df.groupby('date').agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).reset_index()
            
        st = swingTop_bottom(monthly_data) # important levels for each stock
        ohlcCompare = filterdTop(st,df,levelAllowance=0.15)
        levelTapping[symbols]=ohlcCompare
        # print(levelTapping)
        
    except StopIteration:
        errorStockList.append(symbols)
        continue
print(levelTapping)
jasonfile = {k: json.dumps(v) for k,v in levelTapping.items()}
df= pd.DataFrame(list(jasonfile.items()),columns=["stocks","levels"])
df.to_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\{timeframe}\\{watchlist}.csv",index=False)

This function will sort the stocks on the basis of intraday(0.5) or positional(1) and on the basis of taps in descending order

In [14]:
def taps2(watchlist,levelAllow,timeframe):
    print(datetime.datetime.now().time())
    today = date.today()
    now=  datetime.datetime.now()
    instruments = kite.instruments("NSE")
    levelAllowance = levelAllow
    nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
    leveltap = pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\{timeframe}\\{watchlist}.csv")
    leveltap['levels'] = leveltap['levels'].apply(json.loads)
    i=0
    first_sort= {}
    levelTapping ={}
    errorStockList=[]
    for symbols in nifty50['Symbol']:
        try:
            instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)      
            try:
                # ohlc = kite.ohlc('NSE:'+symbols)
                # close_data =  ohlc['NSE:'+symbols]['last_price']
                # print(ohlc)
                # data = kite.historical_data(instrument_token,"2024-12-11","2024-12-11", "day")
                # close_data = data[0]['close']
                ltp = kite.ltp('NSE:'+symbols)
                close_data = ltp['NSE:'+symbols]['last_price']
                # print(close_data)
                # time.sleep(2)
            except:
                print('error')
            addpercentage,subpercentage = percentage(close_data,levelAllowance) #THIS VALUES REPRESENTS THE RANGE FROM LTP
            sortedlist = first_shortlist(symbols,addpercentage,subpercentage,leveltap.iloc[i]['levels'],close_data)
            i+=1
            if sortedlist != None:
                first_sort[instrument_token] = sortedlist
        except StopIteration:
            errorStockList.append(symbols)
            continue
    
    flattened_data = []
    for key,values in first_sort.items():
        for symbol,prices in values.items():
            price_set =  list(prices[0])
            flattened_data.append([key,symbol,price_set,prices[1]])
    first_sorted_dataframe =  pd.DataFrame(flattened_data,columns=['token','symbol','levels','ltp'])
    first_sorted_csv = first_sorted_dataframe.to_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv",index=False)
    csvFile= pd.read_csv('E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv')
    csvFile['levels'] = csvFile['levels'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
    df = csvFile.explode('levels')
    df[['level','taps']] = pd.DataFrame(df['levels'].tolist(), index = df.index)
    df=df.drop(columns='levels')
    df = df.sort_values(by='taps', ascending=False)
    if levelAllowance==1:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_positional_{today.strftime("%d_%m_%y")}.csv',index=False)
    elif levelAllowance==0.5:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_intraday_{today.strftime("%d_%m_%y")}_{now.strftime("%H_%M_%S")}.csv',index=False)
    else:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_custom3_{today.strftime("%d_%m_%y")}.csv',index=False)
        
    print(errorStockList)
    print(len(errorStockList))

In [16]:

def submit():
    watchlist =watchlist_var.get()
    timeframe = timeframe_var.get()
    if range_var.get() == "positional":
        levelAllowance =1
    elif range_var.get() == "intraday":
        levelAllowance=0.5
    else:
        levelAllowance=3
    taps2(watchlist,levelAllowance,timeframe)
    print(f"Watchlist: {watchlist_var.get()}")
    print(f"Timeframe: {timeframe_var.get()}")
    print(f"Range: {range_var.get()}")

# Create the main window
root = tk.Tk()
root.title("Dropdown Selection")

# Create variables for storing the selected values
watchlist_var = tk.StringVar()
timeframe_var = tk.StringVar()
range_var = tk.StringVar()

# Create the dropdowns
watchlist_label = ttk.Label(root, text="Watchlist")
watchlist_label.pack(pady=5)
watchlist_dropdown = ttk.Combobox(root, textvariable=watchlist_var, values=["FNO's", "nifty50", "nifty500","niftyIndices"])
watchlist_dropdown.pack(pady=5)

timeframe_label = ttk.Label(root, text="Timeframe")
timeframe_label.pack(pady=5)
timeframe_dropdown = ttk.Combobox(root, textvariable=timeframe_var, values=["W", "M"])
timeframe_dropdown.pack(pady=5)

range_label = ttk.Label(root, text="Range")
range_label.pack(pady=5)
range_dropdown = ttk.Combobox(root, textvariable=range_var, values=["positional", "intraday","3"])
range_dropdown.pack(pady=5)

# Create the submit button
submit_button = ttk.Button(root, text="Submit", command=submit)
submit_button.pack(pady=20)

# Start the Tkinter event loop
root.mainloop()



10:25:36.525383
222.57
2108.6
787.3
1090.7
773.45
656.4
1815.25
513.5
409.1
67.26
162.14
1267.85
655.3
4529
64.51
647.75
1300.2
231.35
318.75
350.15
147.94
5729.7
1541.85
64.34
4385
7720
64.2
7566.8
1458
2421.4
352.35
108.36
1510
3223.5
6032.15
259
1309
977.3
510.45
952.3
4967.85
123.3
1969.3
985
6884.55
439.5
56.12
764.8
2841.85
239.3
248.78
7353.25
11463.45
829.25
147.2
2519.75
126.05
2819.35
573.3
544
2978.85
1243
4285.35
3044.9
232.53
11272
1274.4
1113.55
408
1761.05
661.8
565
627.45
1466.6
301
1328.35
619.75
2637
275.2
230.23
828.95
100.18
47.61
261.05
5765
1583.05
427.35
560.5
1865.45
2232
687.6
781.8
45.95
7225.6
2949.25
187.51
9008.35
2068.45
172.98
1901.7
928.55
1795.25
55.95
987.05
5394.2
164.79
1229.9
36550
757.85
210.41
805.75
5837
4837.6
408.15
213.49
128.37
58.23
498.9
4663.35
7109.7
881.55
3327.25
2250.9
22.85
321.25
140.15
191.75
559.2
570.55
850.4
2357.8
288.85
2225.05
2930.95
107.43
91.16
2516.9
1758.5
63
2878.45
1853.3
1085.3
2824.25
1343.4
2390.3
795.15
3882.95
707.

In [13]:
def everyHour():
    schedule.every().hour.do(taps)

    while True:
        schedule.run_pending()
        time.sleep(1)

def atTheEnd():
    schedule.every().day.at("13:00").do(taps2)

    while True:
        schedule.run_pending()
        time.sleep(1)

In [13]:
def check():
    print('hello')


def check2():
    print('world')

In [14]:
import threading


t1 =  threading.Thread(target=check)
t2 = threading.Thread(target=check2)

t1.start()
t2.start()

t1.join()
t2.join()

hello
world


In [ ]:
import smtplib
smtp_server= smtplib.SMTP('smtp.gmail.com',587)
smtp_server.ehlo()
smtp_server.starttls()
smtp_server.login('sudipto11bhukta@gmail.com',"")
smtp_server.sendmail("sudipto11bhukta@gmail.com","tradernaman1988@gmail.com","hello from sudipto this is a test mail")
smtp_server.quit()

(220, b'2.0.0 Ready to start TLS')